<a href="https://colab.research.google.com/github/Henrykim1111/tensorflow-developer-certificate/blob/main/%5BTEST%5DTF3_horses_or_humans_type_B_(%EB%AC%B8%EC%A0%9C)%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####################################################
## [중요]
## 2023년 6월 1일 기준
## Google Colab에 설치된 텐서플로우(TensorFlow) 버전은 2.12 버전입니다.
## 시험에 제출 가능한 버전은 2.9.0 버전이기 때문에 버전을 다운그레이드 진행해야 합니다.
## 반드시 아래 코드를 실행하여 버전 다운그레이드 후 모델링을 진행하세요.
## 시험 볼 때는 현재 이 코드는 지워주셔야 합니다.
## 관련하여 궁금하신 점은 슬랙 커뮤니티에 질문 남겨 주세요.
####################################################
import urllib.request

url = 'https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/tfcert.py'
urllib.request.urlretrieve(url, 'tfcert.py')
%run tfcert.py

import tensorflow as tf
import tensorflow_datasets as tfds

print(f'설치 완료 후 TensorFlow 버전: {tf.__version__}')
print(f'설치 완료 후 TensorFlow Datasets 버전: {tfds.__version__}')

## 시험을 위한 버전 확인 ###########
## TensorFlow:          2.9.0 #
## TensorFlow Datasets: 4.6.0 #
###############################

In [1]:
# Question
#
# This task requires you to create a classifier for horses or humans using
# the provided data. Please make sure your final layer is a 1 neuron, activated by sigmoid as shown.
# Please note that the test will use images that are 300x300 with 3 bytes color depth so be sure to design your neural network accordingly

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 3 - Horses Or Humans (Type B)
# val_loss: 0.51 (더 낮아도 안 좋고, 높아도 안 좋음!)
# val_acc: 관계없음
# =================================================== #
# =================================================== #


import urllib.request
import zipfile
import numpy as np
from IPython.display import Image

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16


def solution_model():
    _TRAIN_URL = "https://storage.googleapis.com/download.tensorflow.org/data/horse-or-human.zip"
    _TEST_URL = "https://storage.googleapis.com/download.tensorflow.org/data/validation-horse-or-human.zip"
    urllib.request.urlretrieve(_TRAIN_URL, 'horse-or-human.zip')
    local_zip = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/horse-or-human/')
    zip_ref.close()
    urllib.request.urlretrieve(_TEST_URL, 'validation-horse-or-human.zip')
    local_zip = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_zip, 'r')
    zip_ref.extractall('tmp/validation-horse-or-human/')
    zip_ref.close()

    TRAINING_DIR = 'tmp/horse-or-human/'
    VALIDATION_DIR = 'tmp/validation-horse-or-human/'

    train_datagen = ImageDataGenerator(
        rescale=1/ 255.0,
        rotation_range=10,
        width_shift_range=0.15,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
    )

    validation_datagen = ImageDataGenerator(
        rescale=1/ 255.0,
    )

    train_generator = train_datagen.flow_from_directory(
        TRAINING_DIR,
        target_size=(300, 300),
        batch_size=64,
        class_mode='binary',
    )

    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(300, 300),
        batch_size=64,
        class_mode='binary',
    )

    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(300, 300, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])



    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


    checkpoint_path = "tmp_checkpoint.ckpt"
    checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                                save_weights_only=True,
                                save_best_only=True,
                                monitor='val_loss',
                                verbose=1)

    model.fit(train_generator,
              validation_data=(validation_generator),
              epochs=3,
              callbacks=[checkpoint],
              )

    model.load_weights(checkpoint_path)

    return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
    model = solution_model()
    model.save("TF3-horses-or-humans-type-B.h5")

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/3
17/17 [==============================] - ETA: 0s - loss: 0.8074 - acc: 0.6602
Epoch 1: val_loss improved from inf to 0.78186, saving model to tmp_checkpoint.ckpt
17/17 [==============================] - 42s 2s/step - loss: 0.8074 - acc: 0.6602 - val_loss: 0.7819 - val_acc: 0.6758
Epoch 2/3
17/17 [==============================] - ETA: 0s - loss: 0.3465 - acc: 0.8520
Epoch 2: val_loss did not improve from 0.78186
17/17 [==============================] - 28s 2s/step - loss: 0.3465 - acc: 0.8520 - val_loss: 1.5321 - val_acc: 0.6836
Epoch 3/3
17/17 [==============================] - ETA: 0s - loss: 0.2381 - acc: 0.9075
Epoch 3: val_loss did not improve from 0.78186
17/17 [==============================] - 28s 2s/step - loss: 0.2381 - acc: 0.9075 - val_loss: 2.4905 - val_acc: 0.7227


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
